In [1]:
import os
import sys
import argparse as ap
import numpy as np
import pandas as pd
import datetime

WORKSPACE_DIR = '../..' #!echo $(cd ../../;pwd)
PROJECT_DIR = f'{WORKSPACE_DIR}/barchartapi' #!echo $(cd ../;pwd)
WORKING_DIR = f'{PROJECT_DIR}/barchartapi' #!echo $(cd ./;pwd)
if WORKING_DIR not in sys.path:
    sys.path.append(WORKING_DIR)
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)
if WORKSPACE_DIR not in sys.path:
    sys.path.append(WORKSPACE_DIR)
print(sys.path)
import futures_series as futs


['', '/Library/Frameworks/Python.framework/Versions/3.6/lib/python36.zip', '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6', '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/lib-dynload', '/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages', '/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/IPython/extensions', '/Users/bperlman1/.ipython', '../../barchartapi/barchartapi', '../../barchartapi', '../..']


In [2]:
from importlib import reload
reload(futs)

<module 'futures_series' from '/Users/bperlman1/Documents/billybyte/pyliverisk/barchartapi/barchartapi/futures_series.py'>

Change the contract information below to suit your needs:

In [6]:
temp_folder = './temp_folder'
api_key = open(f'{temp_folder}/paid_api_key.txt','r').read()
commodity = 'GC'

Keep these the same

In [7]:
args = ap.Namespace(
    bar_type = 'minutes',
    interval = 1,
    output_folder = temp_folder,
    endpoint_type = 'paid_url'
)
quarter_commodities = ['ES','GE','GC']

In [8]:
bar_type = args.bar_type
interval = args.interval
output_folder = temp_folder
logger = futs.init_root_logger('logger.log')
endpoint_type = args.endpoint_type

logger.info(f'start')
years = np.linspace(2008,2018,2018-2008+1,dtype=int)
days = 40
month_codes = None
if commodity in quarter_commodities:
    month_codes = ['H','M','U','Z']
    days = 120
logger.info(f'days,month_codes {(days,month_codes)}')
for y in years:
    beg_yyyymm = int(f'{y}01')
    end_yyyymm = int(f'{y}12')
    logger.info(f'begin:{beg_yyyymm}, end:{end_yyyymm}')
    fc = futs.FuturesSeries(api_key, commodity, beg_yyyymm, end_yyyymm, bar_type, 
                            interval, endpoint_type,logger)
    logger.info(f'BEGIN Fetching Data for year: {y}')
    fc.get_contracts(trading_days_to_get=days,month_code_list=month_codes)
    logger.info(f'BEGIN Writing Data for year: {y}')
    fc.write_csv(output_folder)
    logger.info(f'END Writing Data for year: {y}')
logger.info(f'end')



2018-12-21 11:51:45,538 - root - INFO - start
2018-12-21 11:51:45,542 - root - INFO - days,month_codes (120, ['H', 'M', 'U', 'Z'])
2018-12-21 11:51:45,544 - root - INFO - begin:200801, end:200812
2018-12-21 11:51:45,608 - root - INFO - BEGIN Fetching Data for year: 2008
  0%|          | 0/4 [00:00<?, ?it/s]2018-12-21 11:51:45,651 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): ondemand.websol.barchart.com


Barchart OnDemand Client: http://ondemand.websol.barchart.com/


2018-12-21 11:51:45,884 - requests.packages.urllib3.connectionpool - DEBUG - http://ondemand.websol.barchart.com:80 "GET /getHistory.json?maxRecords=60&interval=1&startDate=20080201&symbol=GCH08&type=daily&apikey=e95f76cac84a3c62139dc3502a3e73e7 HTTP/1.1" 200 None
2018-12-21 11:51:45,890 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): ondemand.websol.barchart.com
2018-12-21 11:51:46,124 - requests.packages.urllib3.connectionpool - DEBUG - http://ondemand.websol.barchart.com:80 "GET /getHistory.json?maxRecords=172800&interval=1&startDate=20071128&symbol=GCH08&type=minutes&apikey=e95f76cac84a3c62139dc3502a3e73e7 HTTP/1.1" 200 None
2018-12-21 11:51:46,128 - root - WARNING - GCH08: {'code': 204, 'message': 'No Content'}
 25%|██▌       | 1/4 [00:00<00:01,  1.93it/s]2018-12-21 11:51:46,138 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): ondemand.websol.barchart.com
2018-12-21 11:51:46,354 - requests.packages.urllib

Barchart OnDemand Client: http://ondemand.websol.barchart.com/


2018-12-21 11:51:52,474 - requests.packages.urllib3.connectionpool - DEBUG - http://ondemand.websol.barchart.com:80 "GET /getHistory.json?maxRecords=60&interval=1&startDate=20090201&symbol=GCH09&type=daily&apikey=e95f76cac84a3c62139dc3502a3e73e7 HTTP/1.1" 200 None
2018-12-21 11:51:52,487 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): ondemand.websol.barchart.com
2018-12-21 11:51:52,930 - requests.packages.urllib3.connectionpool - DEBUG - http://ondemand.websol.barchart.com:80 "GET /getHistory.json?maxRecords=172800&interval=1&startDate=20081127&symbol=GCH09&type=minutes&apikey=e95f76cac84a3c62139dc3502a3e73e7 HTTP/1.1" 200 None
 25%|██▌       | 1/4 [00:00<00:02,  1.32it/s]2018-12-21 11:51:53,022 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): ondemand.websol.barchart.com
2018-12-21 11:51:53,772 - requests.packages.urllib3.connectionpool - DEBUG - http://ondemand.websol.barchart.com:80 "GET /getHistory.json?m

...

2018-12-21 11:53:00,473 - requests.packages.urllib3.connectionpool - DEBUG - http://ondemand.websol.barchart.com:80 "GET /getHistory.json?maxRecords=60&interval=1&startDate=20160201&symbol=GCH16&type=daily&apikey=e95f76cac84a3c62139dc3502a3e73e7 HTTP/1.1" 200 None
2018-12-21 11:53:00,480 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): ondemand.websol.barchart.com
2018-12-21 11:53:01,082 - requests.packages.urllib3.connectionpool - DEBUG - http://ondemand.websol.barchart.com:80 "GET /getHistory.json?maxRecords=172800&interval=1&startDate=20151130&symbol=GCH16&type=minutes&apikey=e95f76cac84a3c62139dc3502a3e73e7 HTTP/1.1" 200 None
 25%|██▌       | 1/4 [00:01<00:03,  1.08s/it]2018-12-21 11:53:01,203 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): ondemand.websol.barchart.com
2018-12-21 11:53:01,497 - requests.packages.urllib3.connectionpool - DEBUG - http://ondemand.websol.barchart.com:80 "GET /getHistory.json?m

Barchart OnDemand Client: http://ondemand.websol.barchart.com/


2018-12-21 11:53:10,442 - requests.packages.urllib3.connectionpool - DEBUG - http://ondemand.websol.barchart.com:80 "GET /getHistory.json?maxRecords=60&interval=1&startDate=20170201&symbol=GCH17&type=daily&apikey=e95f76cac84a3c62139dc3502a3e73e7 HTTP/1.1" 200 None
2018-12-21 11:53:10,455 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): ondemand.websol.barchart.com
2018-12-21 11:53:10,958 - requests.packages.urllib3.connectionpool - DEBUG - http://ondemand.websol.barchart.com:80 "GET /getHistory.json?maxRecords=172800&interval=1&startDate=20161129&symbol=GCH17&type=minutes&apikey=e95f76cac84a3c62139dc3502a3e73e7 HTTP/1.1" 200 None
 25%|██▌       | 1/4 [00:00<00:02,  1.08it/s]2018-12-21 11:53:11,096 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): ondemand.websol.barchart.com
2018-12-21 11:53:11,380 - requests.packages.urllib3.connectionpool - DEBUG - http://ondemand.websol.barchart.com:80 "GET /getHistory.json?m

Barchart OnDemand Client: http://ondemand.websol.barchart.com/


2018-12-21 11:53:20,523 - requests.packages.urllib3.connectionpool - DEBUG - http://ondemand.websol.barchart.com:80 "GET /getHistory.json?maxRecords=60&interval=1&startDate=20180201&symbol=GCH18&type=daily&apikey=e95f76cac84a3c62139dc3502a3e73e7 HTTP/1.1" 200 None
2018-12-21 11:53:20,534 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): ondemand.websol.barchart.com
2018-12-21 11:53:21,166 - requests.packages.urllib3.connectionpool - DEBUG - http://ondemand.websol.barchart.com:80 "GET /getHistory.json?maxRecords=172800&interval=1&startDate=20171127&symbol=GCH18&type=minutes&apikey=e95f76cac84a3c62139dc3502a3e73e7 HTTP/1.1" 200 None
 25%|██▌       | 1/4 [00:01<00:03,  1.02s/it]2018-12-21 11:53:21,266 - requests.packages.urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): ondemand.websol.barchart.com
2018-12-21 11:53:21,447 - requests.packages.urllib3.connectionpool - DEBUG - http://ondemand.websol.barchart.com:80 "GET /getHistory.json?m